# Derivation and Abuse of the EKF Method for Real-World Slammage
Kalman Filters are used for recursively estimating the state of a linear system with noise or uncertainty. Extended Kalman Filters (EKF) allow this technique to be applied to nonlinear systems. This facilitates an EKF-based solution to the SLAM problem.


### Preface
This guide page assumes a relative degree of comfort with probability and linear algebra. None of the math here is super nasty but there are a lot of steps so being comfortable with matrix operations (i.e. multiplications, what is a Transposition and why is it used, what is a Jacobian and why is it used), and clearly understanding what an expectation of a distribution is will go a long way. I will briefly summarize these now for the precocious uninitiated:

1. WTF is a Transposition and why is it used?: $\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix}, \quad \mathbf{x}^T = \begin{bmatrix} x_1 & x_2 & x_3 \end{bmatrix}$ Now we can multiply these two vectors together. We can also do the same thing with matrices.

2. WTF is a Jacobian and why is it used?: Special matrix we use to get the the first-order derivative of a vectorial function: $J_f = \begin{bmatrix}
\frac{\partial f_1(x)}{\partial x_1} & \frac{\partial f_1(x)}{\partial x_2} & \cdots & \frac{\partial f_1(x)}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial f_n(x)}{\partial x_1} & \frac{\partial f_n(x)}{\partial x_2} & \cdots & \frac{\partial f_n(x)}{\partial x_n}
\end{bmatrix}, \quad J_ff(\cdot) = \nabla f(\cdot)$

3. WTF is an expectation and why is it used?: Most likely value of a Random Variable. Think of it as the weighted average across all outcomes, with each outcome multiplied by it's respective probability: $\mathbb{E}[X] = \int_{-\infty}^{\infty} x \, p(x) \, dx$ or $\mathbb{E}[X] = \sum_{x} x \, p(x)$ depending on if the RV is continuous or discrete.

I also want to make the disclaimer that while robotics is used as the context for explaining why some of the math below works out the way it does, it's not the only way. EKF can be used for other things as well, but this whole piece is centered around explaining it in the context of SLAM, and so robotics becomes the chosen medium.
### Part 1 - The Derivation

#### 1.0 - Preliminaries

For now we will just focus on the robot's state, $x_k$. We will define the state vector distribution as some function of the previous state distribution, $f(x_{k-1})$, with zero-mean gaussian noise, $w_{k-1}$, added to it to account for jump uncertainty (for example, uncertainty in the path-planning algo). We will define our observation vector distribution, $z_k$, as some function of the current state, $f(x_k)$, with zero-mean gaussian noise, $v_{k}$, added to it to account for observation uncertainty (sensor noise). Our cookage manifests the following two equations:


>$$
>(1) \quad x_k = f(x_{k-1}) + w_{k-1} 
>$$
>$$(2) \quad z_k = f(x_k) + v_{k-1} 
>$$

We define the initial state, $x_0$, as a random vector with known mean $\mu_0 = \mathbb{E}[x_0]$ and covariance $P_0 = \mathbb{E}\left[(x_0 - \mu_0)(x_0 - \mu_0)^T\right]$ 

From here, we are going to make a few assertions:

1. Noise vector distributions have a zero-valued mean (I explained the intuition behind this in the previous part): $\quad\mathbb{E}[w_k] = 0, \quad\mathbb{E}[v_k] = 0$  

2. The two types of noise vectors never correlate with each other: $\quad\mathbb{E}[w_k v_k^T j] = 0 \quad \forall k, j$  

3. Neither noise vector has any correlation with the start point: $\quad\mathbb{E}[w_k x_0^T] = 0 \quad \mathbb{E}[v_k x_0^T] = 0 \quad \forall k$  

4. Neither noise vector has any correlation whatsoever with any of its' predecessors or successors: $\quad\mathbb{E}[w_k w_j^T] = 0, \quad \mathbb{E}[v_k v_j^T] = 0 \quad  \forall k \neq j$

5. Vector variance (some sources use the term covariance but I think this is silly, because covariance with oneself is just variance) is represented with the following two matrices: $\quad\mathbb{E}[w_k w_k^T] = Q_k, \quad \mathbb{E}[v_k v_k^T] = R_k$


We also make the assumption that functions $f(\cdot)$ and $h(\cdot)$ as well as their first-order derivatives are continuous on the given domain.

To summarize, we have made a laundry-list of assertions to ensure that no Brimless Yankee activities will break what we do next. Dimensionality may be implictly known by now it is nonetheless summarized below:


$$
x_k, \quad n \times 1 \quad\text{-- State vector at time step} \space k \newline
$$
$$
w_k, \quad n \times 1 \quad\text{-- Process noise vector} \newline
$$
$$
z_k, \quad m \times 1 \quad\text{-- Observation vector at time step} \space k \newline
$$
$$
v_k, \quad m \times 1 \quad\text{-- Measurement noise vector} \newline
$$
$$
f(\cdot), \quad n \times 1 \quad\text{-- Process nonlinear vector function} \newline
$$
$$
h(\cdot), \quad m \times 1 \quad\text{-- Observation nonlinear vector function} \newline
$$
$$
\mathbb{Q_k}, \quad n \times n \quad\text{-- Process noise covariance matrix} \newline
$$
$$
\mathbb{R_k}, \quad m \times m \quad\text{-- Measurement noise covariance matrix} \newline
$$


> [!NOTE]
>
>In most (all?) real-world use-cases as far as SLAM is concerned, $n$ should always occupy a value between 1 and 3 inclusive. However, in order not to anger our high-dimensional overlords (the same mf's responsible for various lighthearted glitch-in-the-matrix activities such as ensuring that one runs into some very specific individual at very specific and unflattering times in a way and at a frequency that probabilistically just doesn't make sense), we are keeping $n$ abstractly defined. As you may imagine, $m$ is abstractly defined because it depends on the specifics of how the sensor suite collects data.

#### 1.1 - Model Forecast Step (Time Update)

This is analogous to the "Time Update" part of our generalized SLAM algorithm. In the beginning, the only information have is the mean, $\mu_0$. We use this to obtain our initial optimal estimate, $x_0^a$ and variance, $\mathbb{P_0}$ in the following manner:

>$$
>(3) \quad x_0^a = \mu_0 = \mathbb{E}[x_0]
>$$
>$$
>(4) \quad \mathbb{P_0} = \mathbb{E}[(x_0 - x_0^a)(x_0 - x_0^a)^T]
>$$

This is intuitive for our initial optimal estimate because we would expect it to be at the starting point that is by definition the most likely. We don't condition on any observation for this step because we operate under the assumption that $\mu_0$ is obtained either through prior contextual knowledge, or an initial guess. Of course, that intial knowledge/guess could be obtained (at least in part) through sensor readings, but then it wouldn't really make sense to condition it on those same sensor readings thereafter. For peace of mind, we will assume that our overlords gift us this information via high-dimensional carrier pigeon, and any sensor readings taken at this location thereafter would only add impurity to the divine estimate. We will not pain ourselves with the specifics of how it was obtained any further.


We are only gifted one high-dimensional carrier piegon, however. This means that, assuming our sensor data is on average more accurate than our odometry data, the following states' ideal estimates should incorporate conditioning from the sensor data in order to be ideal.

>$$
>(5) \quad x_{k-1}^a \equiv \mathbb{E}[x_{k-1} | z_{k-1}]
>$$

Now, for reasons that are not yet obvious, let's say we want to get Taylor's Version (Taylor Aproximation) of $f(x_{k-1})$. We can do that as follows:

$$f(x_{k-1}) \equiv f(x_{k-1}^a) + \mathbb{J_f}(x_{k-1} - x_{k-1}^a) + H.O.T.$$

Where the Jacobian, $\mathbb{J_f}$, is defined as:

$$
J_f = \begin{bmatrix}
\frac{\partial f_1(x)}{\partial x_1} & \frac{\partial f_1(x)}{\partial x_2} & \cdots & \frac{\partial f_1(x)}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\
\frac{\partial f_n(x)}{\partial x_1} & \frac{\partial f_n(x)}{\partial x_2} & \cdots & \frac{\partial f_n(x)}{\partial x_n}
\end{bmatrix}
$$

In the spirit of Taylor, we simplify for the masses:

>$$(6) \quad f(x_{k-1}) \approx f(x_{k-1}^a) + \mathbb{J_f}e_{k-1}$$

Where $e_{k-1} \equiv x_{k-1} - x_{k-1}^a$ and the higher-order terms are dropped because they have negligible effect on the result. Now we have a convenient way of estimating $f(x_{k-1})$ knowing only $f(x_{k-1}^a)$, $x_{k-1}^a$, and $x_{k-1}$. As you may expect, at this point this is completely useless because we dont know $x_{k-1}$, and we also wouldn't know $x_{k-1}^a$ unless $k-1 = 0$. As you may expect in addition to the previously expected, all following math in this document is dedicated towards ultimately rectifying this.

Now what we want to do is figure out how to get the most accurate possible 'time update' value for $x_k$, which we denote as $x_k^f$. To do this we consider the Markov (Bumbling) process from part 0:
1. Move to a new state $x_k$ as a function of the previous state $f(x_{k-1})$
2. Make an observation $z_k$ that is probably more accurate than our odometry data.

From this, it's relatively easy to see that the best way we could get an estimate for $x_k^f$ would be to take the expectation of $f(x_{k-1})$ conditioned on $z_{k-1}$. Doing so yields the following:

$$
x_k^f = \mathbb{E}[f(x_{k-1}) | z_{k-1}] \approx \mathbb{E}[(f(x_{k-1}^a) + \mathbb{J_f}(x_{k-1}^a)e_{k-1}) | z_{k-1}]
$$
$$
= f(x_{k-1}^a) + \mathbb{J_f(x_{k-1}^a)}\mathbb{E}[e_{k-1} | z_{k-1}]
$$
>$$
>(7) \quad \therefore x_k^f \approx f(x_{k-1}^a)
>$$


The expectation of $f(x_{k-1}^a) | z_{k-1}$ is $f(x_{k-1}^a)$ because $x_{k-1}^a$ is itself an expectation conditioned on $z_{k-1}$.

The expectation of $e_{k-1} | z_{k-1}$ is zero because we expect, on average, a delta of zero between our ideal estimate and our actual value. As with $x_{k-1}^a$, this delta will already have the observation conditioning baked in. If this isn't intuitive, review unbiased estimators.

>[!NOTE]
>
>Of course, sensor bias does exist in the real world but for now we will assume that either it doesn't, or that we can compensate for it in a deterministic manner.

Now we have a decent enough approximation for $x_k^f$, which is itself a function of our best possible approximation for $x_{k-1}$, but this causes a new problem to appear: $x_k$ is a distribution, not a single number like $x_k^f$. In addition to this, it's not like we really knew what our $x_{k-1}$ was in the first place because it itself was also a distribution, we just took our best guess, $x_{k-1}^a$. 

This means that to get our current position distribution, $x_k$, we have to add noise back into the equation. This noise represents the uncertainty from our previous state estimations that is propagated through the process. Our final equation looks like this:

$$
x_k = x_k^f + w_{k-1}
$$
>$$ 
>(8) \quad \therefore x_k^f \approx f(x_{k-1}^a) + w_{k-1}
>$$

The noise has a zero-mean as mentioned earlier. You can rationalize this to yourself by asserting that on average, our previous state estimations will more or less be accurate due to the fact that our estimator (driven by unbiased odometry and sensor data) is unbiased, and will therefore have an overall average deviation of 0 from the true value. 


We now have a formula for $x_k^f$ which lets us expand on two terms that we will make use of later. The first of these terms is forecast error, $e_k$:
>$$
>(9) \quad e_k^f \equiv x_k - x_k^f
>$$
$$
= \mathbf{f}(\mathbf{x}_{k-1}) + \mathbf{w}_{k-1} - \mathbf{f}(\mathbf{x}_{k-1}^a) 
$$
$$
\approx \mathbf{J}_f(\mathbf{x}_{k-1}^a) \mathbf{e}_{k-1} + \mathbf{w}_{k-1}
$$


The second of these terms is our forecast error variance, $\mathbb{P_k^f}$:
>$$
>(10) \quad \mathbb{P_k^f} \equiv \mathbb{E}[e_k^f(e_k^f)^T]
>$$
$$
= J_f(x_{k-1}^a)\mathbb{E}[e_{k-1}e_{k-1}^T]J_f^T(x_{k-1}^a) + E[w_{k-1}w_{k-1}^T]
$$
$$
= J_f(x_{k-1}^a)P_{k-1}J_f^T(x_{k-1}^a) + Q_{k-1}
$$

Once again, we assume that any linear noise has an expectation of zero.


The next thing we have to do is take $x_k$ and condition on our observation $z_k$ in order to obtain the ideal estimate used for the next time step of the process: $x_k^a$



#### 1.2 - Data Assimilation Step (Measurement Update)
We now seek to condition our forecast value, $x_k^f$, in order to obtain our ideal unbiased estimate (ideal in the least squares sense), $x_k^a$, which estimates $x_k$ and is used to drive the upcoming forecast in the next cycle of our Markov process. To do this, we assert that $x_k^a$ is some linear combination of $x_k^f$ and $z_k$, let:

$$
x_k^a = a + \mathbb{K_k}z_k
$$

where $a$ contains $x_k^f$ and some other stuff.


From the notion that our estimate is unbiased, we can assert that the expected error is zero:














### Part 2 - The Abuse
Now we abuse our freshly derived EKF method in order to solve our SLAM problem.

